<a href="https://colab.research.google.com/github/Ricardo371/s-and-p-500-companies/blob/master/killer_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Description: We are going to optimize a portfolio based on the S&P500 listed companies.
#import the libraries
import pandas as pd
import numpy as np
import requests

In [5]:
#Load the data. I got the data using this program I made. YOu can find it here https://github.com/Ricardo371/s-and-p-500-companies/blob/master/Stock_closingprice_data.ipynb
#It is around 11 years of data. 
#from google.colab import files
#files.upload()


In [6]:
# Now we store the data.
#In retrospective I could have loaded the data from the files tab. 
#Much faster than thru the line code.
df = pd.read_csv('prices_S&P500.csv')

#I had the column of the date labeled as 'Symbol' so I use rename to change it to 'date'.
df= df.rename(columns={'Symbol': 'Date'})

#Set the date as the index
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

#Remove the date colum
df.drop(columns=['Date'], axis=1, inplace=True)

In [7]:
#Get the assets / tickers
assets = df.columns

In [8]:
#Here we're going to use a package called PyPortfolioOpt to optimize our portfolio.
%pip install PyPortfolioOpt

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 1.0MB 11.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.10-cp36-cp36m-linux_x86_64.whl size=2683431 sha256=1efbba09065c78fd21d1137903c2a2f03a2dd7b5e66fbc4b2f126fcd5de22475
  Stored in directory: /root/.cache/pip/wheels/64/d6/c9/9295499aefa5af80018eaabffd09216170a1349b45551fb8d5
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [9]:
#Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [10]:
#Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns.
mu = expected_returns.mean_historical_return(df)
S = risk_models.CovarianceShrinkage(df).ledoit_wolf()

In [11]:
#Optimize for the maximal Sharpe ratio.
ef = EfficientFrontier(mu, S) #This will create the efficient frontier object.

In [12]:
weights = ef.max_sharpe()

/usr/local/lib/python3.6/dist-packages/cvxpy/problems/problem.py:1126: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


In [13]:
clean_weights = ef.clean_weights()
print(clean_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('MMM', 0.0), ('AOS', 0.0), ('ABT', 0.0), ('ABBV', 0.0), ('ABMD', 0.0), ('ACN', 0.0), ('ATVI', 0.0), ('ADBE', 0.0), ('AAP', 0.0), ('AMD', 0.0), ('AES', 0.0), ('AFL', 0.0), ('A', 0.0), ('APD', 0.0), ('AKAM', 0.0), ('ALK', 0.0), ('ALB', 0.0), ('ARE', 0.0), ('ALXN', 0.0), ('ALGN', 0.0), ('ALLE', 0.0), ('LNT', 0.0), ('ALL', 0.0), ('GOOGL', 0.0), ('GOOG', 0.0), ('MO', 0.0), ('AMZN', 0.0), ('AMCR', 0.0), ('AEE', 0.0), ('AAL', 0.00011), ('AEP', 0.0), ('AXP', 0.0), ('AIG', 0.0001), ('AMT', 0.0), ('AWK', 0.0), ('AMP', 0.0001), ('ABC', 0.0), ('AME', 0.0), ('AMGN', 0.0), ('APH', 0.0), ('ADI', 0.0), ('ANSS', 0.0), ('ANTM', 0.0), ('AON', 0.0), ('APA', 0.00013), ('AAPL', 0.0), ('AMAT', 0.0), ('APTV', 0.0), ('ADM', 0.0), ('ANET', 0.0), ('AJG', 0.0), ('AIZ', 0.0), ('T', 0.0), ('ATO', 0.0), ('ADSK', 0.0), ('ADP', 0.0), ('AZO', 0.0), ('AVB', 0.0), ('AVY', 0.0), ('BKR', 0.0001), ('BLL', 0.0), ('BAC', 0.0001), ('BAX', 0.0), ('BDX', 0.0), ('BRK.B', 0.0), ('BBY', 0.0), ('BIO', 0.0), ('BIIB', 0.

(2.202665866269034, 0.15596538542164995, 13.994553088611497)

In [14]:
%pip install pulp

     |████████████████████████████████| 40.6MB 112kB/s 


In [28]:
#Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

portfolio_val = 1000
latest_prices = get_latest_prices(df)
latest_prices = latest_prices.fillna(0)
weights = clean_weights

In [29]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = portfolio_val)


In [30]:
allocation, leftover = da.lp_portfolio()
print('Discrete allocation: ', allocation)
print('Funds remaining: $', leftover)

Discrete allocation:  {'APA': 1, 'CCL': 1, 'CARR': 23, 'DVN': 1, 'HST': 1, 'KEY': 1, 'MRO': 1, 'NOV': 1, 'RF': 1, 'FTI': 1}
Funds remaining: $ 0.5300321578980629


In [37]:
#Create a function to get the companies name
def get_company_name(symbol):
  url = 'http://d.yimg.com/autoc.finance.yahoo.com/autoc?query='+symbol+'&region=1&lang=en'
  result = requests.get(url).json()
  for r in result['ResultSet']['Result']:
    if r['symbol']==symbol:
      return r['name']

In [38]:
#Store the name into a list
company_name =[]
for symbol in allocation:
  company_name.append(get_company_name(symbol))

In [39]:
#Get the discrete allocation values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [40]:
#Create a dataframe for the portfolio
portfolio_df = pd.DataFrame(columns=['Company_name', 'Company_ticker', 'Discrete_val_'+str(portfolio_val)])

In [41]:
portfolio_df['Company_name']= company_name
portfolio_df['Company_ticker']= allocation
portfolio_df['Discrete_val_'+str(portfolio_val)]= discrete_allocation_list

In [42]:
portfolio_df

,Company_name,Company_ticker,Discrete_val_1000
0,Apache Corporation,APA,1
1,Carnival Corporation & plc,CCL,1
2,Carrier Global Corporation,CARR,23
3,Devon Energy Corporation,DVN,1
4,"Host Hotels & Resorts, Inc.",HST,1
5,KeyCorp,KEY,1
6,Marathon Oil Corporation,MRO,1
7,NOV Inc.,NOV,1
8,Regions Financial Corporation,RF,1
9,TechnipFMC plc,FTI,1
